In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import MultiPolygon
from shapely.geometry import Point, MultiPoint
import os
import sys
import pysal as ps
import matplotlib.pyplot as plt

C:\Users\szej01\AppData\Local\Continuum\anaconda3\envs\geo2\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


In [2]:
os.getcwd()
sys.path.append(os.path.realpath('..'))

In [3]:
filename = r"..\input_data\Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016.csv"
physician_puf16_df = pd.read_csv(filename)

C:\Users\szej01\AppData\Local\Continuum\anaconda3\envs\geo2\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Pulling out the subset of HCPCS code


In [4]:
lungcancerscreening = pd.Series(['G0297'])

In [5]:
physician_puf16_df['lungcancerscreening'] = physician_puf16_df['HCPCS Code'].isin(lungcancerscreening)

In [6]:
physician_puf16_df['lungcancerscreening'].mean()

0.00019979627162246513

In [7]:
physician_puf16_df['HCPCS Code'][physician_puf16_df.lungcancerscreening == True].head(10)

2720     G0297
3144     G0297
4602     G0297
4603     G0297
13921    G0297
13981    G0297
35969    G0297
36931    G0297
44910    G0297
46518    G0297
Name: HCPCS Code, dtype: object

In [8]:
physician_puf16_df['lungcancerscreening'].value_counts()

False    9712955
True        1941
Name: lungcancerscreening, dtype: int64

In [9]:
LungCancerScreening_puf16 = pd.DataFrame(physician_puf16_df.loc[physician_puf16_df['lungcancerscreening'] == True])

In [10]:
LungCancerScreening_puf16.columns

Index(['National Provider Identifier',
       'Last Name/Organization Name of the Provider',
       'First Name of the Provider', 'Middle Initial of the Provider',
       'Credentials of the Provider', 'Gender of the Provider',
       'Entity Type of the Provider', 'Street Address 1 of the Provider',
       'Street Address 2 of the Provider', 'City of the Provider',
       'Zip Code of the Provider', 'State Code of the Provider',
       'Country Code of the Provider', 'Provider Type',
       'Medicare Participation Indicator', 'Place of Service', 'HCPCS Code',
       'HCPCS Description', 'HCPCS Drug Indicator', 'Number of Services',
       'Number of Medicare Beneficiaries',
       'Number of Distinct Medicare Beneficiary/Per Day Services',
       'Average Medicare Allowed Amount', 'Average Submitted Charge Amount',
       'Average Medicare Payment Amount',
       'Average Medicare Standardized Amount', 'lungcancerscreening'],
      dtype='object')

In [11]:
collapsed_LungCancerScreening_puf16 = gpd.GeoDataFrame(LungCancerScreening_puf16.groupby('National Provider Identifier').agg({
    'Number of Services': sum
})).reset_index() 

In [12]:
collapsed_LungCancerScreening_puf16.count()

National Provider Identifier    1860
Number of Services              1860
dtype: int64

In [13]:
filepath =  r"..\intermediate\LungCancerScreening_puf16.csv"
save_out = collapsed_LungCancerScreening_puf16[['National Provider Identifier','Number of Services']]
save_out.to_csv(filepath)

In [15]:
collapsed_LungCancerScreening_puf16[collapsed_LungCancerScreening_puf16['National Provider Identifier'] == 1003027376].head()

,National Provider Identifier,Number of Services
2,1003027376,75.0
